In [1]:
# To do
# Load beers in
# Run nltk on the beers to get frequency counts

# Get list of keywords for each beer
# Store data in easily reachable way
# Run tf-idf to get list of keywords
# Compare each list of words for each beer by clustering. PCA and not kmeans?
# Get clusters and store them in an easily accessible way
# Input a beer and get the keywords for that beer OR input a list of keywords to match to.

import re
import json
import nltk
import numpy as np
import pandas as pd
import string
from collections import OrderedDict
from nltk.corpus import stopwords

In [2]:
beers = pd.read_csv('../beers/beerlist.csv')
print('number of beers = '+(str(len(beers))))
beers['words']

number of beers = 4813


0       ['pours', 'dark', 'brow', 'almost', 'looks', '...
1       ['medium', 'capped', 'version', 'chalice', 'op...
2       ['pleasant', 'surprise', 'quite', 'good', 'rel...
3       ['bottled', 'april', '117th', 'day', 'hoping',...
4       ['12oz', 'bottle', 'first', 'expierence', 'bra...
                              ...                        
4808    ['bottle', 'poured', 'goblet', 'clear', 'beaut...
4809    ['pours', 'typical', 'clear', 'golden', 'color...
4810    ['red', 'light', 'haze', 'lots', 'champagne', ...
4811    ['clear', 'redamber', 'color', 'light', 'flow'...
4812    ['pours', 'golden', 'orange', 'extremely', 'fi...
Name: words, Length: 4813, dtype: object

In [3]:
def pre_process(text):
    # lowercase
    text=text.lower()
    #remove tags
    text=re.sub("<!--?.*?-->","",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    return text

beers['words'] = beers['words'].apply(lambda x:pre_process(x)) 

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

cv = TfidfVectorizer(min_df=0.05, max_df=0.85,stop_words='english')
word_count_vector = cv.fit_transform(beers['words'])
print(np.shape(word_count_vector))
list(cv.vocabulary_.keys())

In [ ]:
beers['words'][0]

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
 
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [ ]:
# Choose a random beer and test this shit
randbeer = np.random.randint(0, len(beers))
beers['words'][randbeer]

# Forgot to account for duplicates. Also there's a better way to do this
beer = beers[beers['name'] == beers['name'][randbeer]]
beerword = beer['words']
beerword = beerword.apply(lambda x:pre_process(x))
test = beerword.tolist()

In [ ]:
# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()
 
# get the document that we want to extract keywords from
doc = test[0]
 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
 
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,10)
 
# now print the results
#print("\n=====Doc=====")
#print(doc)
print(beer['name'])
print("\n===Keywords===")
for k in keywords:
    print(k,keywords[k])


In [ ]:
wordlist = beer['words'].tolist()[0].split(' ')
words = [wordlist[t].translate(wordlist[t].maketrans("","", string.punctuation)) for t in range(len(wordlist))]
words = [x.strip(' ') for x in words]
sr = stopwords.words('english')
clean_tokens = words[:]
for token in words:
    if token in stopwords.words('english'):
        clean_tokens.remove(token)
freq = nltk.FreqDist(clean_tokens)
for key,val in freq.items():
    print(str(key) + ':' + str(val))

In [ ]:
freq.plot(20, cumulative=False)

In [4]:
names = beers['name'][:1500]
words = beers['words'][:1500]

In [5]:
from nltk.stem.snowball import SnowballStemmer
stopwords = nltk.corpus.stopwords.words('english')
stemmer = SnowballStemmer('english')

# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        filtered_tokens.append(token)
    return filtered_tokens

In [6]:
totalvocab_stemmed = []
totalvocab_tokenized = []
for i in range(len(words)):
    allwords_stemmed = tokenize_and_stem(words[i])
    totalvocab_stemmed.extend(allwords_stemmed)
    allwords_tokenized = tokenize_only(words[i])
    totalvocab_tokenized.extend(allwords_tokenized)
    if i%100 == 0:
        print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


In [7]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print('there are '+str(vocab_frame.shape[0])+' items in vocab_frame')

there are 5615383 items in vocab_frame


In [8]:
vocab_frame.head()

,words
pour,pours
dark,dark
brow,brow
almost,almost
look,looks


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.85, min_df=0.1, stop_words='english', use_idf=True, tokenizer=tokenize_and_stem)

tfidf_matrix = tfidf_vectorizer.fit_transform(words) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

In [ ]:
terms = tfidf_vectorizer.get_feature_names()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
dist = 1 - cosine_similarity(tfidf_matrix)

In [17]:
from sklearn.cluster import KMeans

num_clusters = 5

km = KMeans(n_clusters=num_clusters)
%time km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

CPU times: user 15min 59s, sys: 4.63 s, total: 16min 4s
Wall time: 15min 25s


In [24]:
from sklearn.externals import joblib

#uncomment the below to save your model 
#since I've already run my model I am loading from the pickle

#joblib.dump(km,  'doc_cluster.pkl')

km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [28]:
beers = { 'name': names, 'words': words, 'cluster': clusters}

frame = pd.DataFrame(beers, index = [clusters] , columns = ['name', 'words', 'cluster'])

NameError: name 'ABV' is not defined

In [26]:
frame['cluster'].value_counts() #number of films per cluster (clusters from 0 to 4)

2    458
0    385
1    264
3    260
4    133
Name: cluster, dtype: int64

In [29]:
print("Top terms per cluster:")
print()
#sort cluster centers by proximity to centroid
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

for i in range(num_clusters):
    print("Cluster %d words:" % i, end='')
    
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        print(' %s' % vocab_frame.ix[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    print() #add whitespace
    print() #add whitespace
    
    print("Cluster %d titles:" % i, end='')
    for title in frame.ix[i]['title'].values.tolist():
        print(' %s,' % title, end='')
    print() #add whitespace
    print() #add whitespace
    
print()
print()



Top terms per cluster:

Cluster 0 words:

/home/steven/anaconda2/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  # Remove the CWD from sys.path while we load stuff.


NameError: name 'terms' is not defined